1) Inicialização: criar uma população inicial com N anticorpos;

2) Avaliação: calcule o fitness de cada elemento;

3) Seleção Clonal e Expansão: Selecione os n anticorpos de maior fitness e gere clones destes anticorpos de maneira proporcional ao fitness -- quanto maior o fitness, maior o número de clones, e vice-versa;

4) Maturação e atividade: aplique mutação sobre os clones segundo uma taxa inversamente proporcional ao fitness. Então, avaliea população de clones  eforme uma nova população com os n melhores clones;

5) Metadinâmica: substitua uma porcentagem p de anticorpos de menor fitness em população por novos gerados aleatoriamente. Este procedimento pode ser feito em periodos de T iterações;

6) Ciclo: repita os passos 2 a 5 até que um determinado critério de parada seja atingido.

# CLONALG Description

In artificial immune systems, clonal selection algorithms are a class of algorithms inspired by the clonal selection theory of acquired immunity that explains how B and T lymphocytes improve their response to antigens over time called affinity maturation. These algorithms focus on the Darwinian attributes of the theory where selection is inspired by the affinity of antigen-antibody interactions, reproduction is inspired by cell division, and variation is inspired by somatic hypermutation. Clonal selection algorithms are most commonly applied to optimization and pattern recognition domains, some of which resemble parallel hill climbing and the genetic algorithm without the recombination operator $[2]$.

<img src="figures/clonalg_pseudocoge.png" alt="Drawing" style="width: 700px;"/>

# Implementation

The demonstration problem is an istance of a continuous function optimization that seeks $min\ f(x)$ where $f\ =\ \sum_{i=1}^{n} x_i^2$, $-5.0 \le x_i \le 5.0$ and $n = 3$. The optimal solution for this basin function is $(v_0, ..., v_{n-1}) = 0.0$ $[1]$.

In [9]:
# Third-party libraries
import numpy as np
from numpy import absolute
from numpy.random import uniform, choice
from random import randint

In [28]:
def affinity(p_i):
    return np.sum(np.power(p_i, 2))

In [18]:
# Inputs
b_lo, b_up = (-5, 5)

pop_size = 100
selection_size = 10
problem_size = 3
clone_factor = 0.1
random_cells_num = 2
max_gens = 100

In [19]:
population = [uniform(low=b_lo, high=b_up, size=problem_size) for x in range(pop_size)]

In [32]:
population[:5]

[array([-2.34878124, -4.10090711, -0.99208912]),
 array([-3.6731006 ,  0.34962855, -3.5504332 ]),
 array([ 1.68208678, -2.70311155, -2.84454385]),
 array([-0.98420836,  0.71007125, -1.99212365]),
 array([-2.82354647, -0.29405737, -1.68410738])]

In [29]:
pop_affinity = [affinity(p_i) for p_i in population]

In [31]:
pop_affinity[:5]

[23.31845321453554,
 26.219484080545104,
 18.22765766490641,
 5.441423918269415,
 10.89510209830063]

# Sources 

[[1] Clonal Selection Algorithm](http://www.cleveralgorithms.com/nature-inspired/immune/clonal_selection_algorithm.html).

[[2] Clonal Selection Algorithm - Wikipedia](https://en.wikipedia.org/wiki/Clonal_selection_algorithm).